# PRIVACY POLICY ANALYSER BOT 

### step 1 :

In [1]:
# Cell 1: Install dependencies from requirements.txt
import os


# Install
!pip install -r requirements.txt

print("All dependencies installed successfully!")

All dependencies installed successfully!
